In [2]:
!pip install numpy==1.16.6
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=3408386 sha256=7c3e5093bf3839cd9e9e7bb912e4d9c579989a64f7a9d7733becde5bba6469b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-4qjk5lqh/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tifffile 2022.3.16 requires numpy>=1.19.2, but you have numpy 1.16.6 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.6 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have n

In [4]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 31.4 MB/s eta 0:00:00


In [2]:
#encoding={'Benign':0, 'FTP-BruteForce':1,'DoS attacks-GoldenEye':2, 'DoS attacks-Slowloris':3,'DoS attacks-SlowHTTPTest':4, 'DoS attacks-Hulk':5,"DDOS attack-HOIC":6,"Bot":7,"DDOS attack-LOIC-UDP":8}

In [5]:
from river import stream
from river import preprocessing
from river import compose
from river import metrics
from river import stats
from river import evaluate
import pandas as pd
from river.feature_selection import PoissonInclusion
from river.imblearn import HardSamplingClassifier
from river.tree import HoeffdingAdaptiveTreeClassifier
from imblearn.metrics import geometric_mean_score
from river.metrics import ClassificationReport

In [6]:
data = pd.read_csv("preprocessed_multiclass_stream.csv")

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [8]:
stream_data = stream.iter_pandas(train.drop("Label", axis = 1), y = train["Label"])

In [9]:
test_data = stream.iter_pandas(test.drop("Label", axis = 1), y = test["Label"])

In [10]:
def construct_pipeline(model):
    return compose.Pipeline(
                PoissonInclusion(p=0.4), # feature selection to reduce training time
                HardSamplingClassifier( # imblearn wrapper to solve class imbalance
                            classifier=model, 
                            p=0.1,
                            size=40,
                        )
                )

In [11]:
HAT_model = HoeffdingAdaptiveTreeClassifier()

In [12]:
model=construct_pipeline(HAT_model)

In [13]:
#model = compose.Pipeline(
    #preprocessing.StandardScaler(),
    # linear_model.LogisticRegression()
    #HoeffdingAdaptiveTreeClassifier())

for x, y in stream_data:
    model = model.learn_one(x, y)      # make the model learn   

In [14]:
Acc_metric = metrics.Accuracy();
Rec_metric = metrics.Recall();
Pre_metric = metrics.Precision();
F1_metric = metrics.F1();

def get_metric(Acc, rec, pre, f1 ):
  Acc, rec, pre, f1 = Acc, rec, pre, f1
  return([Acc, rec, pre, f1])

predictions = []
true_y = []
for x, y in test_data:
    y_pred = model.predict_one(x) # make a prediction
    predictions.append(y_pred)
    true_y.append(y)
    Acc_metric = Acc_metric.update(y, y_pred)
    Rec_metric = Rec_metric.update(y, y_pred)
    Pre_metric = Pre_metric.update(y, y_pred)
    F1_metric = F1_metric.update(y, y_pred)
    model = model.learn_one(x, y)
print(get_metric(Acc_metric, Rec_metric, Pre_metric, F1_metric))  

[Accuracy: 85.65%, Recall: 84.70%, Precision: 68.12%, F1: 75.51%]


In [15]:
report = ClassificationReport()
for yt,yp in zip(true_y, predictions):
  report = report.update(yt,yp)
print(report)
print("geometric mean score :",geometric_mean_score(true_y, predictions, average='weighted'))

           Precision   Recall   F1       Support  
                                                  
       0      90.84%   51.25%   65.53%      8283  
       1      68.12%   84.70%   75.51%      5202  
       2      86.16%   97.74%   91.59%      8281  
       3      95.66%   97.74%   96.69%      2077  
       4      69.04%   46.37%   55.48%      3843  
       5      91.44%   61.36%   73.44%      3168  
       6      99.47%   99.94%   99.70%     16782  
       7      78.50%   98.15%   87.23%      1446  
       8       0.85%    0.31%    0.45%       327  
                                                  
   Macro      75.56%   70.84%   71.74%            
   Micro      85.65%   85.65%   85.65%            
Weighted      86.16%   85.65%   84.51%            

                 85.65% accuracy                  
geometric mean score : 0.9126195798985223
